In [1]:
import os
import cv2
import numpy as np
import random

# ------------------------------
# Configuration
# ------------------------------
IMAGE_DIR = r"C:\Users\DELL\Downloads\Apple Leaf Diseases Image Dataset of ICAR-CITH\Apple Leaf Diseases Image Dataset of ICAR-CITH\Apple Diseases Images\Apple Diseases Images\Apple Apple scab"
IMG_SIZE = (224, 224)

In [2]:
# Image Cleaning
# ------------------------------
def clean_scab_image(img):
    img = cv2.resize(img, IMG_SIZE)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hsv = hsv.astype(np.float32) / 255.0
    return hsv

In [3]:
def augment_scab_image(img):
    aug = img.copy()

    if random.random() < 0.5:
        angle = random.randint(-20, 20)
        h, w = aug.shape[:2]
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
        aug = cv2.warpAffine(aug, M, (w, h))

    if random.random() < 0.5:
        factor = random.uniform(0.8, 1.2)
        aug = np.clip(aug * factor, 0, 1)

    if random.random() < 0.3:
        aug = cv2.GaussianBlur(aug, (3, 3), 0)

    return aug

In [4]:
# Scab Segmentation (HSV)
# ------------------------------
def apple_scab_mask(hsv_img):
    hsv_uint8 = (hsv_img * 255).astype(np.uint8)

    lower = np.array([15, 40, 20])
    upper = np.array([50, 255, 180])

    mask = cv2.inRange(hsv_uint8, lower, upper)

    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    return mask

In [5]:
# Edge Detection
# ------------------------------
def edge_detection(hsv_img):
    gray = cv2.cvtColor((hsv_img * 255).astype(np.uint8), cv2.COLOR_HSV2BGR)
    gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    return edges

In [6]:
# Boundary Detection
# ------------------------------
def boundary_detection(mask, original_img):
    contours, _ = cv2.findContours(
        mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    output = original_img.copy()
    cv2.drawContours(output, contours, -1, (0, 255, 0), 2)
    return output

In [7]:
import random

# ------------------------------
# Select 2 random images
# ------------------------------
all_images = [
    f for f in os.listdir(IMAGE_DIR)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
]

sampled_images = random.sample(all_images, 2)

# IMAGE PREPROCESSING
# ------------------------------
for filename in sampled_images:
    path = os.path.join(IMAGE_DIR, filename)

    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    cleaned = clean_scab_image(img)
    augmented = augment_scab_image(cleaned)
    mask = apple_scab_mask(augmented)
    edges = edge_detection(augmented)
    boundaries = boundary_detection(
        mask,
        cv2.resize(img, IMG_SIZE)
    )

    cv2.imshow("Original", cv2.resize(img, IMG_SIZE))
    cv2.imshow("Scab Mask", mask)
    cv2.imshow("Edges", edges)
    cv2.imshow("Scab Boundaries", boundaries)

    cv2.waitKey(0)

cv2.destroyAllWindows()
